In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx
from networkx.algorithms import bipartite

from tqdm.notebook import tqdm

In [ ]:
# save each artist as json
for i, artist in tqdm(enumerate(final_artists)):
    with open(f"artists/artist{i}.json", "w") as outfile:
        json.dump(artist, outfile) 

In [ ]:
# save each track as json
for i, track in tqdm(enumerate(more_tracks)):
    with open(f"tracks/track{i}.json", "w") as outfile:
        json.dump(track, outfile)

In [64]:
artist_jsons = !cd "artists"; find *.json

artists_list = []
for file in artist_jsons:
    data = pd.read_json(f"artists/{file}", lines=True)
    artists_list.append(data)
    
artists = pd.concat(artists_list, ignore_index=True)
artists

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
0,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 149}",[],https://api.spotify.com/v1/artists/3hTMyEnHbZ2...,3hTMyEnHbZ2fTRod9AzqEo,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Bianca Sings,24,artist,spotify:artist:3hTMyEnHbZ2fTRod9AzqEo
1,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 43429}",[florida rap],https://api.spotify.com/v1/artists/09drtGC393C...,09drtGC393C6jGa05qYccB,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kolyon,37,artist,spotify:artist:09drtGC393C6jGa05qYccB
2,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 50}",[],https://api.spotify.com/v1/artists/6c10vP5GpwZ...,6c10vP5GpwZ2j8CiuuWeS4,[],Choppa Wop,32,artist,spotify:artist:6c10vP5GpwZ2j8CiuuWeS4
3,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 93588}","[modern ska punk, punk, ska, ska punk, skate p...",https://api.spotify.com/v1/artists/31v7ijnAMVL...,31v7ijnAMVL9V7Uzr1sFcI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Suicide Machines,49,artist,spotify:artist:31v7ijnAMVL9V7Uzr1sFcI
4,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 69111}","[dubstep, edm, electro house, electronic trap]",https://api.spotify.com/v1/artists/21AUdblPrTR...,21AUdblPrTRzkvJn8FGrlk,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Nitti Gritti,63,artist,spotify:artist:21AUdblPrTRzkvJn8FGrlk
...,...,...,...,...,...,...,...,...,...,...
4589,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 2339}",[],https://api.spotify.com/v1/artists/53EmQSV3tRY...,53EmQSV3tRYsAlmM6kElwL,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",DaBoyDame,37,artist,spotify:artist:53EmQSV3tRYsAlmM6kElwL
4590,{'spotify': 'https://open.spotify.com/artist/7...,"{'href': None, 'total': 46285}","[chicago rap, drill, vapor trap]",https://api.spotify.com/v1/artists/7AO1qih2zQe...,7AO1qih2zQeI7WYTQxC5fQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Tadoe,53,artist,spotify:artist:7AO1qih2zQeI7WYTQxC5fQ
4591,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 647043}","[brooklyn drill, new york drill, nyc rap]",https://api.spotify.com/v1/artists/1tG7s7S4sq2...,1tG7s7S4sq2eFFW0QZyLbm,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Sheff G,75,artist,spotify:artist:1tG7s7S4sq2eFFW0QZyLbm
4592,{'spotify': 'https://open.spotify.com/artist/5...,"{'href': None, 'total': 2467}",[],https://api.spotify.com/v1/artists/55ny2DqS7aB...,55ny2DqS7aBuL4hinLeBfG,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",yoitsCrash,27,artist,spotify:artist:55ny2DqS7aBuL4hinLeBfG


In [95]:
copy_of_artists = artists.copy()
# tracks.head()
main_genre_for_artist = np.zeros(len(artists), dtype=list)
for i, artist_genres in enumerate(copy_of_artists['genres'].to_list()):
    if (len(artist_genres) == 0):
        main_genre_for_artist[i] = "other"
    elif ("chicago rap" in artist_genres):
        main_genre_for_artist[i] = 'chicago rap'
    else:
        main_genre_for_artist[i] = artist_genres[0]
unique_genres, unique_genre_counts = np.unique(main_genre_for_artist, return_counts=True)

In [105]:
unique_genres, unique_genre_counts = np.unique(main_genre_for_artist, return_counts=True)
chicago_rap_count = unique_genres[unique_genre_counts == unique_genre_counts.max()]
unique_genre_counts[unique_genre_counts == unique_genre_counts.max()]/unique_genre_counts.sum()

array([0.41663039])

In [136]:
other_artists = [len(i)==0 for i in artists['genres']]
# other_artists
copy_of_artists2 = copy_of_artists.drop(copy_of_artists[other_artists].index)
copy_of_artists2['genres'] = main_genre_for_artist[main_genre_for_artist != 'other']

In [ ]:
num_tracks = !ls -l tracks/ | wc -l
track_jsons = [f"track{i}.json" for i in range(int(num_tracks[0].strip()))]
# print(track_jsons)
tracks_list = []
artists_list = []

for file in track_jsons:
    data = pd.read_json(f"tracks/{file}", lines=True)
    tracks_list.append(data)
    
tracks = pd.concat(tracks_list, ignore_index=True)
tracks

In [85]:
copy_of_tracks = tracks.copy()
# tracks.head()
artists_per_track = np.zeros(len(tracks), dtype=list)
for i, track in enumerate(tracks['artists'].to_list()):
    artists_per_track[i] = [artist['name'] for artist in track]
copy_of_tracks['artists'] = artists_per_track
copy_of_tracks

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,is_local,name,preview_url,track_number,type,uri
0,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,155440,False,{'spotify': 'https://open.spotify.com/track/79...,https://api.spotify.com/v1/tracks/792ywURG9Ayc...,792ywURG9AycMjt9I1TEgD,False,Everytime I Feel the Spirit,https://p.scdn.co/mp3-preview/eaa0c04b30543f95...,1,track,spotify:track:792ywURG9AycMjt9I1TEgD
1,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,255240,False,{'spotify': 'https://open.spotify.com/track/57...,https://api.spotify.com/v1/tracks/57rGxFsw7gBi...,57rGxFsw7gBiGOyuqYXzq7,False,Go Down Moses,https://p.scdn.co/mp3-preview/8333db97e6969f19...,2,track,spotify:track:57rGxFsw7gBiGOyuqYXzq7
2,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,226040,False,{'spotify': 'https://open.spotify.com/track/0A...,https://api.spotify.com/v1/tracks/0AuzsPNc3GSU...,0AuzsPNc3GSUHVpfLFJqGZ,False,"Children, Go Where I Send Thee",https://p.scdn.co/mp3-preview/f6a87fcf1d683339...,3,track,spotify:track:0AuzsPNc3GSUHVpfLFJqGZ
3,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,223026,False,{'spotify': 'https://open.spotify.com/track/47...,https://api.spotify.com/v1/tracks/47jySDLe5rBz...,47jySDLe5rBzXnj2sHzBVG,False,Little David Play On Your Harp,https://p.scdn.co/mp3-preview/85928da90fb8608b...,4,track,spotify:track:47jySDLe5rBzXnj2sHzBVG
4,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,169680,False,{'spotify': 'https://open.spotify.com/track/5K...,https://api.spotify.com/v1/tracks/5KIH6pSKTNrD...,5KIH6pSKTNrDdsHJDmckGi,False,"Ride On, King Jesus",https://p.scdn.co/mp3-preview/9b3f362b8e576d28...,5,track,spotify:track:5KIH6pSKTNrDdsHJDmckGi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45019,[Bon Iver],"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,329279,False,{'spotify': 'https://open.spotify.com/track/7v...,https://api.spotify.com/v1/tracks/7vE7w0zMSYRR...,7vE7w0zMSYRR29fIZBTiyp,False,Blindsided,https://p.scdn.co/mp3-preview/fd135167b431f87b...,5,track,spotify:track:7vE7w0zMSYRR29fIZBTiyp
45020,[Bon Iver],"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,186132,False,{'spotify': 'https://open.spotify.com/track/6a...,https://api.spotify.com/v1/tracks/6afJTmreY8lM...,6afJTmreY8lMnG9F4Uo1tM,False,Creature Fear,https://p.scdn.co/mp3-preview/8002c944ece0008e...,6,track,spotify:track:6afJTmreY8lMnG9F4Uo1tM
45021,[Bon Iver],"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,116919,False,{'spotify': 'https://open.spotify.com/track/0X...,https://api.spotify.com/v1/tracks/0XltgXfjLVvx...,0XltgXfjLVvxY0intOijLq,False,Team,https://p.scdn.co/mp3-preview/9cb3b36f94689606...,7,track,spotify:track:0XltgXfjLVvxY0intOijLq
45022,[Bon Iver],"[AD, AL, AM, AT, AZ, BA, BE, BG, BY, CH, CW, C...",1,220732,False,{'spotify': 'https://open.spotify.com/track/64...,https://api.spotify.com/v1/tracks/64jVNNTwT0Nd...,64jVNNTwT0NdQbqqUIa8gi,False,For Emma,https://p.scdn.co/mp3-preview/86359673ac343684...,8,track,spotify:track:64jVNNTwT0NdQbqqUIa8gi


In [2]:
# generating test data
songs_data = {'id': [1, 2, 3, 4], 'title': ['songone', 'songtwo', 'songthree', 'songfour'], 'artists': [['kanye'], ['kanye', 'drake'], ['drake', 'durk'], ['drake', 'durk']]}
songs = pd.DataFrame(data=songs_data)
artists_data = {'id': ['a', 'b', 'c'], 'name': ['kanye', 'drake', 'durk'], 'genre': ['chicago rap', 'pop rap', 'chicago rap']}
artists = pd.DataFrame(data=artists_data)

In [3]:
songs

,id,title,artists
0,1,songone,[kanye]
1,2,songtwo,"[kanye, drake]"
2,3,songthree,"[drake, durk]"
3,4,songfour,"[drake, durk]"


In [139]:
copy_of_tracks.head()

,artists,available_markets,disc_number,duration_ms,explicit,external_urls,href,id,is_local,name,preview_url,track_number,type,uri
0,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,155440,False,{'spotify': 'https://open.spotify.com/track/79...,https://api.spotify.com/v1/tracks/792ywURG9Ayc...,792ywURG9AycMjt9I1TEgD,False,Everytime I Feel the Spirit,https://p.scdn.co/mp3-preview/eaa0c04b30543f95...,1,track,spotify:track:792ywURG9AycMjt9I1TEgD
1,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,255240,False,{'spotify': 'https://open.spotify.com/track/57...,https://api.spotify.com/v1/tracks/57rGxFsw7gBi...,57rGxFsw7gBiGOyuqYXzq7,False,Go Down Moses,https://p.scdn.co/mp3-preview/8333db97e6969f19...,2,track,spotify:track:57rGxFsw7gBiGOyuqYXzq7
2,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,226040,False,{'spotify': 'https://open.spotify.com/track/0A...,https://api.spotify.com/v1/tracks/0AuzsPNc3GSU...,0AuzsPNc3GSUHVpfLFJqGZ,False,"Children, Go Where I Send Thee",https://p.scdn.co/mp3-preview/f6a87fcf1d683339...,3,track,spotify:track:0AuzsPNc3GSUHVpfLFJqGZ
3,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,223026,False,{'spotify': 'https://open.spotify.com/track/47...,https://api.spotify.com/v1/tracks/47jySDLe5rBz...,47jySDLe5rBzXnj2sHzBVG,False,Little David Play On Your Harp,https://p.scdn.co/mp3-preview/85928da90fb8608b...,4,track,spotify:track:47jySDLe5rBzXnj2sHzBVG
4,[The Boys Choir Of Harlem],"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",1,169680,False,{'spotify': 'https://open.spotify.com/track/5K...,https://api.spotify.com/v1/tracks/5KIH6pSKTNrD...,5KIH6pSKTNrDdsHJDmckGi,False,"Ride On, King Jesus",https://p.scdn.co/mp3-preview/9b3f362b8e576d28...,5,track,spotify:track:5KIH6pSKTNrDdsHJDmckGi


In [4]:
artists

,id,name,genre
0,a,kanye,chicago rap
1,b,drake,pop rap
2,c,durk,chicago rap


In [137]:
copy_of_artists2.head()

,external_urls,followers,genres,href,id,images,name,popularity,type,uri
1,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 43429}",florida rap,https://api.spotify.com/v1/artists/09drtGC393C...,09drtGC393C6jGa05qYccB,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Kolyon,37,artist,spotify:artist:09drtGC393C6jGa05qYccB
3,{'spotify': 'https://open.spotify.com/artist/3...,"{'href': None, 'total': 93588}",modern ska punk,https://api.spotify.com/v1/artists/31v7ijnAMVL...,31v7ijnAMVL9V7Uzr1sFcI,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",The Suicide Machines,49,artist,spotify:artist:31v7ijnAMVL9V7Uzr1sFcI
4,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 69111}",dubstep,https://api.spotify.com/v1/artists/21AUdblPrTR...,21AUdblPrTRzkvJn8FGrlk,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Nitti Gritti,63,artist,spotify:artist:21AUdblPrTRzkvJn8FGrlk
6,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 23466400}",colombian pop,https://api.spotify.com/v1/artists/0EmeFodog0B...,0EmeFodog0BfCgMzAIvKQp,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Shakira,86,artist,spotify:artist:0EmeFodog0BfCgMzAIvKQp
7,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 165803}",chutney,https://api.spotify.com/v1/artists/6wxP7SSzfvi...,6wxP7SSzfvi21Cnl8JicdQ,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Machel Montano,58,artist,spotify:artist:6wxP7SSzfvi21Cnl8JicdQ


In [140]:
# creating the initial bipartite graph
# I need to do this for each year

B = nx.Graph()

# adding nodes for songs and artists

B.add_nodes_from((copy_of_tracks['name']).to_list(), bipartite=0)
for idx, artist in copy_of_artists2.iterrows():
#     print(f"added node '{artist['name']}' with atrribute '{artist['genre']}'")
    B.add_node(artist['name'], genre=artist['genres'])
    
print(B.nodes.data())
    
# adding an edge if an artist is on a song
for index, song in copy_of_tracks.iterrows():
    for index, artist in copy_of_artists2.iterrows():
        if (artist['name'] in song['artists']):
#             print(f'add edge between {song["title"]} and {artist["name"]}')
            B.add_edges_from([(song["name"], artist["name"])])

print(B.edges.data())
# print(bipartite.is_bipartite(B))
# G = bipartite.weighted_projected_graph(B, [1, 3])

[('Everytime I Feel the Spirit', {'bipartite': 0}), ('Go Down Moses', {'bipartite': 0}), ('Children, Go Where I Send Thee', {'bipartite': 0}), ('Little David Play On Your Harp', {'bipartite': 0}), ('Ride On, King Jesus', {'bipartite': 0}), ('Ezekiel Saw De Wheel', {'bipartite': 0}), ("I've Been Buked", {'bipartite': 0}), ('If I Got My Ticket, Can I Ride', {'bipartite': 0}), ('Mary Was the Queen of Galilee', {'bipartite': 0}), ('Amazing Grace', {'bipartite': 0}), ('I Can Tell the World', {'bipartite': 0}), ('Precious Lord', {'bipartite': 0}), ('Kumbaya My Lord', {'bipartite': 0}), ('Oh Freedom', {'bipartite': 0}), ('This Is The Day', {'bipartite': 0}), ('Know Your Life Will change This Christmas', {'bipartite': 0}), ('We Got Shoes', {'bipartite': 0}), ("We're So Glad U Came", {'bipartite': 0}), ('Fix Me jesus', {'bipartite': 0}), ('This Is The Day (remix)', {'bipartite': 0}), ('We Got Shoes (remix)', {'bipartite': 0}), ("We're So Glad U Came (remix)", {'bipartite': 0}), ('Fix Me Jesus (

[('Everytime I Feel the Spirit', 'The Boys Choir Of Harlem', {}), ('Go Down Moses', 'The Boys Choir Of Harlem', {}), ('Children, Go Where I Send Thee', 'The Boys Choir Of Harlem', {}), ('Little David Play On Your Harp', 'The Boys Choir Of Harlem', {}), ('Ride On, King Jesus', 'The Boys Choir Of Harlem', {}), ('Ezekiel Saw De Wheel', 'The Boys Choir Of Harlem', {}), ("I've Been Buked", 'The Boys Choir Of Harlem', {}), ('If I Got My Ticket, Can I Ride', 'The Boys Choir Of Harlem', {}), ('Mary Was the Queen of Galilee', 'The Boys Choir Of Harlem', {}), ('Amazing Grace', 'The Boys Choir Of Harlem', {}), ('I Can Tell the World', 'The Boys Choir Of Harlem', {}), ('Precious Lord', 'The Boys Choir Of Harlem', {}), ('Precious Lord', "Chicago Children's Choir", {}), ('Kumbaya My Lord', 'The Boys Choir Of Harlem', {}), ('Oh Freedom', 'The Boys Choir Of Harlem', {}), ('This Is The Day', 'The Boys Choir Of Harlem', {}), ('This Is The Day', 'Fred Hammond', {}), ('Know Your Life Will change This Chri

In [141]:
# projecting the bipartite graph onto the artists
song_nodes, artist_nodes = bipartite.sets(B)
G = bipartite.weighted_projected_graph(B, artist_nodes)
print("nodes")
print(G.nodes.data())
print("\nedges")
print(G.edges.data())

AmbiguousSolution: Disconnected graph: Ambiguous solution for bipartite sets.

In [ ]:
nx.write_gexf(G, "first_test_graph.gexf")

In [142]:
nx.write_gexf(B, "first_test_graph.gexf")

False
